Załadowanie bibliotek i utworzenie obiektu Login

In [ ]:
#!import ./../load.ipynb

Utworzenie statycznego ActivitySource dla rozszerzenia. Należy pamiętać o włączeniu go w appsettings.json (Telemetry:TracesSources).




In [ ]:
using System.Diagnostics;

static ActivitySource source = new ActivitySource("BestSonetaAddon");

Utworzenie Activity dla procesu przetwarzania towarów. 

Do spana dodane zostały tagi z nazwą użytkownika i liczbą towarów z EAN.

Zarejestowano także event informujący o wystąpieniu nieprawidłowego towaru (bez EAN).

Należy pamiętać, że w przypadku braku nasłuchu na podstawione ActivitySource metoda StartActivity zwróci null.


In [ ]:
using Soneta.Towary;

using(var activity = source.StartActivity("Load products"))
{
  using(var session = Login.CreateSession("Pobieranie listy towarów"))
  {
    activity?.AddTag("UserName", Login.UserName);
    int validCount = 0;
    foreach(var towar in session.GetTowary().Towary.WgNazwy){
      if (string.IsNullOrWhiteSpace(towar.EAN))
        activity?.AddEvent(new ActivityEvent($"Product {towar.Nazwa} without EAN"));
      else
        validCount++;
      WriteLine(towar.Nazwa);
    }
    activity?.AddTag("ValidCount", validCount);
  }
}

Kod przetwarzający towary analogiczny jak powyższy, ale zamiast tworzenia nowego Activity, dodajemy Event do nadrzędnego Activity (jeśli jest utworzony).

In [ ]:
using Soneta.Towary;

using(var session = Login.CreateSession("Pobieranie listy towarów"))
{
    int validCount = 0;
    foreach(var towar in session.GetTowary().Towary.WgNazwy){
        if (string.IsNullOrWhiteSpace(towar.EAN))
            Activity.Current?.AddEvent(new ActivityEvent($"Product {towar.Nazwa} without EAN"));
        else
            validCount++;
        WriteLine(towar.Nazwa);
    }
}

Wyczyszczenie obiektu Login

In [ ]:
Login?.Dispose();